In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/py-crack/Segmentation/Ground truth/173.png
/kaggle/input/py-crack/Segmentation/Ground truth/248.png
/kaggle/input/py-crack/Segmentation/Ground truth/94.png
/kaggle/input/py-crack/Segmentation/Ground truth/236.png
/kaggle/input/py-crack/Segmentation/Ground truth/340.png
/kaggle/input/py-crack/Segmentation/Ground truth/48.png
/kaggle/input/py-crack/Segmentation/Ground truth/350.png
/kaggle/input/py-crack/Segmentation/Ground truth/314.png
/kaggle/input/py-crack/Segmentation/Ground truth/227.png
/kaggle/input/py-crack/Segmentation/Ground truth/238.png
/kaggle/input/py-crack/Segmentation/Ground truth/61.png
/kaggle/input/py-crack/Segmentation/Ground truth/222.png
/kaggle/input/py-crack/Segmentation/Ground truth/330.png
/kaggle/input/py-crack/Segmentation/Ground truth/278.png
/kaggle/input/py-crack/Segmentation/Ground truth/303.png
/kaggle/input/py-crack/Segmentation/Ground truth/37.png
/kaggle/input/py-crack/Segmentation/Ground truth/231.png
/kaggle/input/py-crack/Segmentation

In [2]:

# IMPORTS & DEVICE


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [3]:
# 2. DATA PATHS & TRANSFORMS 


data_root = "/kaggle/input/py-crack/Classification"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

full_dataset = datasets.ImageFolder(data_root, transform=transform)
num_classes = len(full_dataset.classes)
print("Classes:", full_dataset.classes)

# Check overall class distribution
print("Full dataset distribution:", Counter(full_dataset.targets))

Classes: ['With crack', 'Without crack']
Full dataset distribution: Counter({0: 369, 1: 200})


In [4]:
#  TRAIN & VALIDATION SPLIT


train_ratio = 0.8
train_size = int(train_ratio * len(full_dataset))
val_size   = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)


Train size: 455, Val size: 114


In [5]:
#  CUSTOM CNN MODEL 


class CrackCNN(nn.Module):
    def __init__(self, num_classes):
        super(CrackCNN, self).__init__()

        self.features = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),        

            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),        

            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),        

    
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),        
        )

        
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  

        self.classifier = nn.Sequential(
            nn.Flatten(),             
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)   # final fully connected layer
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

model = CrackCNN(num_classes).to(device)
print(model)

CrackCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [6]:
#  LOSS, CLASS WEIGHTS, OPTIMIZER, HYPERPARAMETERS


labels_np = np.array(full_dataset.targets)
class_counts = Counter(full_dataset.targets)
print("Class counts:", class_counts)

class_weights_np = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels_np),
    y=labels_np
)
class_weights = torch.tensor(class_weights_np, dtype=torch.float).to(device)
print("Class weights:", class_weights)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)  

num_epochs    = 20      
min_epochs    = 5       
patience      = 5       
best_val_loss = np.inf
patience_counter = 0

train_losses = []
val_losses   = []

Class counts: Counter({0: 369, 1: 200})
Class weights: tensor([0.7710, 1.4225])


In [7]:
#  TRAINING LOOP + VALIDATION + EARLY STOPPING


for epoch in range(num_epochs):

    model.train()
    running_train_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    
    model.eval()
    running_val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    val_losses.append(epoch_val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"- Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

    
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        patience_counter = 0
        best_state_dict = model.state_dict()   # save best model
    else:
        patience_counter += 1

    if (epoch + 1) >= min_epochs and patience_counter >= patience:
        print("Early stopping triggered.")
        break


model.load_state_dict(best_state_dict)


Epoch [1/20] - Train Loss: 0.5650 | Val Loss: 0.7133
Epoch [2/20] - Train Loss: 0.4329 | Val Loss: 0.6478
Epoch [3/20] - Train Loss: 0.3584 | Val Loss: 0.4140
Epoch [4/20] - Train Loss: 0.3523 | Val Loss: 0.3969
Epoch [5/20] - Train Loss: 0.3539 | Val Loss: 0.2904
Epoch [6/20] - Train Loss: 0.2877 | Val Loss: 0.2486
Epoch [7/20] - Train Loss: 0.2420 | Val Loss: 0.2228
Epoch [8/20] - Train Loss: 0.2390 | Val Loss: 0.2040
Epoch [9/20] - Train Loss: 0.2120 | Val Loss: 0.2737
Epoch [10/20] - Train Loss: 0.2364 | Val Loss: 0.2214
Epoch [11/20] - Train Loss: 0.1712 | Val Loss: 0.1783
Epoch [12/20] - Train Loss: 0.1835 | Val Loss: 0.2473
Epoch [13/20] - Train Loss: 0.1827 | Val Loss: 0.1422
Epoch [14/20] - Train Loss: 0.1447 | Val Loss: 0.2685
Epoch [15/20] - Train Loss: 0.1916 | Val Loss: 0.1351
Epoch [16/20] - Train Loss: 0.1528 | Val Loss: 0.1629
Epoch [17/20] - Train Loss: 0.1592 | Val Loss: 0.2436
Epoch [18/20] - Train Loss: 0.1477 | Val Loss: 0.1210
Epoch [19/20] - Train Loss: 0.1814 | 

<All keys matched successfully>

In [8]:
#  EVALUATION 


model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

accuracy = (y_true == y_pred).mean()
print(f"\nFinal Accuracy on Validation Set: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=full_dataset.classes))


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))



Final Accuracy on Validation Set: 0.9211

Classification Report:
               precision    recall  f1-score   support

   With crack       0.96      0.92      0.94        74
Without crack       0.86      0.93      0.89        40

     accuracy                           0.92       114
    macro avg       0.91      0.92      0.91       114
 weighted avg       0.92      0.92      0.92       114


Confusion Matrix:
[[68  6]
 [ 3 37]]
